# 4. Inference

In [1]:
import sys, os
sys.path.append("../../..")

import pathSetting
import pandas as pd

from Clust.clust.ML.common.common import p1_integratedDataSaving as p1
from Clust.clust.ML.tool import data as ml_data
from Clust.clust.ML.tool import scaler as ml_scaler

import torch
#import main_regression as mr
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"{device}" " is available.")

/Users/kim-youngkee/Documents/CLUSTER/.clust/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


cpu is available.


## 4-1. (just for test) Data Preparation 

In [2]:
# 0. pick only one data
DataMeta = p1.read_json_data(pathSetting.DataMetaPath)
dataList =  list(DataMeta.keys())
# dataX

dataName_X = dataList[2]
dataSaveMode_X = DataMeta[dataName_X]["integrationInfo"]["DataSaveMode"]

# datay
dataName_y = dataList[3]
dataSaveMode_y = DataMeta[dataName_y]["integrationInfo"]["DataSaveMode"]
dataFolderName = "data_integrated_result"
current = os.getcwd()
dataFolderPath = os.path.join(current, dataFolderName)

window_num =144
data_X = ml_data.get_saved_integrated_data(dataSaveMode_X, dataName_X, dataFolderPath)[:window_num]

In [3]:
from Clust.clust.ingestion.mongo import mongo_client as mongo_client
from Clust.setting import influx_setting_KETI as ins

mongo_client = mongo_client.MongoClient(ins.CLUSTMetaInfo2)

db_name = 'model'
collection_name = 'meta'

all_model_meta = mongo_client.get_all_document(db_name, collection_name)

In [4]:
db_name = 'model'
collection_name = 'meta'
search = {'modelName': 'EnergyLSTMRegression'}
model_meta= mongo_client.get_document_by_json(db_name, collection_name, search)
model_meta = model_meta[1]

In [5]:
model_meta

{'author': 'KETI',
 'modelName': 'EnergyLSTMRegression',
 'modelVersion': 0.1,
 'trainDataInfo': {'startTime': '2016-01-11',
  'endTime': '2016-04-15',
  'dataInfo': [['life_indoor_environment', 'humidityTrain_10min'],
   ['life_indoor_environment', 'temperatureTrain_10min'],
   ['weather_outdoor_environment', 'belgiumChieverseAirportTrain_10min']],
  'processParam': {'refine_param': {'removeDuplication': {'flag': False},
    'staticFrequency': {'flag': False, 'frequency': None}},
   'outlier_param': {'certainErrorToNaN': {'flag': False},
    'unCertainErrorToNaN': {'flag': False, 'param': {}}},
   'imputation_param': {'flag': False,
    'imputation_method': [],
    'totalNonNanRatio': 80}},
  'integration_freq_sec': 600,
  'cleanParam': 'NoClean',
  'DataSaveMode': 'CSV'},
 'featureList': ['RH_1',
  'RH_2',
  'RH_3',
  'RH_4',
  'RH_5',
  'RH_6',
  'RH_7',
  'RH_8',
  'RH_9',
  'T1',
  'T2',
  'T3',
  'T4',
  'T5',
  'T6',
  'T7',
  'T8',
  'T9',
  'Press_mm_hg',
  'RH_out',
  'T_out'

In [6]:


X_scaler_file_path = model_meta['files']['XScalerFile']["filePath"]
y_scaler_file_path = model_meta['files']['yScalerFile']["filePath"]
model_file_path = model_meta['files']['modelFile']["filePath"]

feature_list = model_meta["featureList"]
target = model_meta["target"]
scaler_param = model_meta["scalerParam"]
model_method = model_meta["model_method"]
train_parameter = model_meta["trainParameter"]

# Scaling Test Input
input_X, scaler_X = ml_scaler.get_scaled_test_data(data_X[feature_list], X_scaler_file_path, scaler_param)
scaler_y = ml_scaler.get_scaler_file(y_scaler_file_path)

In [8]:
# 4. Inference
from Clust.clust.ML.regression_YK.inference import RegressionInference as RI

ri = RI()
ri.set_param(train_parameter)
ri.set_model(model_method, model_file_path)
ri.set_data(input_X, window_num)
preds = ri.inference()


Start inference

** Dimension of result for inference dataset = (1,)


In [10]:
if scaler_param =='scale':
    base_df_for_inverse = pd.DataFrame(columns=target, index=range(len(preds)))
    base_df_for_inverse[target] = preds
    prediction_result = pd.DataFrame(scaler_y.inverse_transform(base_df_for_inverse), columns=target, index=base_df_for_inverse.index)
else:
    prediction_result = pd.DataFrame(data={"value": preds}, index=range(len(preds)))

In [11]:
prediction_result

,value
0,11.656283
